In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
import os
from utils import mrd
import importlib

import dask
import dask.dataframe as dd
from dask.distributed import Client, progress

In [2]:
# Import and subset day of ES4 data (DOY 282)

base_dir = "C:/Users/moo90/Box/data/materhorn/raw_data/ES/ES4/raw_20hz/"
data = pd.read_csv(os.path.join(base_dir, "DPG-UoU_ES4_3063_20Hz_FluxTower_20121007193013_20121009184326.txt"), 
                   skiprows=[0,2,3], header=0, na_values=["NAN"], parse_dates=[0], index_col=[0])
 



In [31]:
importlib.reload(mrd)

<module 'utils.mrd' from 'c:\\Users\\moo90\\Box\\git\\fluxtopo\\fluxtopo\\utils\\mrd.py'>

In [4]:
#### Old, multi-index way (ran into issues when reading back data)

def interp_func(df, limit=5):
    return df.interpolate(method='time', limit=limit)



#data_282 = data.loc['2012-10-08'].map_partitions(interp_func)

# Create height multiindex
data_282.columns = pd.MultiIndex.from_tuples([('_'.join(col.split('_')[:-1]), 
                                               float(col.split('_')[-1])) for col in data_282.columns], 
                                               names=['var','height'])

def doy_mrd(data, num_hrs, doy, pair_dict, dt=.05):
    """
    Create multiindex df of mrd data from day of year
    """
    # NOTE: need to fix timescales on here
    heights = data['Ux'].columns.values
    n_heights = heights.shape[0]
    mrd_len = mrd.find_pow2(int(num_hrs * (1/dt) * 3600))

    mrd_multiindex = pd.MultiIndex.from_product([list(pair_dict), heights.tolist(), ['t', 'd', 'd_accum']], names=['var','height','values'])
    hourly_multiindex = pd.MultiIndex.from_product([np.arange(24), np.arange(mrd_len)], names=['hour', 'mrd_index'])
    mrd_df = pd.DataFrame(index=np.arange(24), columns=mrd_multiindex)

    for p in pair_dict.keys():
        print(p)

        for i,h in enumerate(heights):
            
            temp_data = data.loc[:,(slice(None),h)]

            # Calculate MRDs for each hour
            for hour in np.arange(24):

                temp_hr_data = temp_data.loc[temp_data.index.hour == hour].interpolate()
                temp_T, temp_d = mrd.mrd_numba(temp_hr_data[pair_dict[p][0]].values.flatten(), temp_hr_data[pair_dict[p][1]].values.flatten())

                mrd_df.loc[(hour), (p, h, 't')] = temp_T * dt
                mrd_df.loc[(hour), (p, h, 'd')] = temp_d
                mrd_df.loc[(hour), (p, h, 'd_accum')] = np.cumsum(temp_d)

    return mrd_df

In [24]:
## New, shiny way of starting out with xarray format and then writing out MRDs to netcdf

def interp_func(df, limit=5):
    return df.interpolate(method='time', limit=limit)

def ts_csv_to_xarray(df):
    
    mrd_ds = 
    return



#data_282 = data.loc['2012-10-08'].map_partitions(interp_func)

# Create height multiindex
data_282.columns = pd.MultiIndex.from_tuples([('_'.join(col.split('_')[:-1]), 
                                               float(col.split('_')[-1])) for col in data_282.columns], 
                                               names=['var','height'])



def doy_mrd(data, num_hrs, doy, pair_dict, dt=.05):
    """
    Create multiindex df of mrd data from day of year
    """
    # NOTE: need to fix timescales on here
    heights = data['Ux'].columns.values
    n_heights = heights.shape[0]
    mrd_len = mrd.find_pow2(int(num_hrs * (1/dt) * 3600))

    mrd_multiindex = pd.MultiIndex.from_product([list(pair_dict), heights.tolist(), ['t', 'd', 'd_accum']], names=['var','height','values'])
    hourly_multiindex = pd.MultiIndex.from_product([np.arange(24), np.arange(mrd_len)], names=['hour', 'mrd_index'])
    mrd_df = pd.DataFrame(index=np.arange(24), columns=mrd_multiindex)

    for p in pair_dict.keys():
        print(p)

        for i,h in enumerate(heights):
            
            temp_data = data.loc[:,(slice(None),h)]

            # Calculate MRDs for each hour
            for hour in np.arange(24):

                temp_hr_data = temp_data.loc[temp_data.index.hour == hour].interpolate()
                temp_T, temp_d = mrd.mrd_numba(temp_hr_data[pair_dict[p][0]].values.flatten(), temp_hr_data[pair_dict[p][1]].values.flatten())

                mrd_df.loc[(hour), (p, h, 't')] = temp_T * dt
                mrd_df.loc[(hour), (p, h, 'd')] = temp_d
                mrd_df.loc[(hour), (p, h, 'd_accum')] = np.cumsum(temp_d)

    return mrd_df

[]


In [32]:
# dict of name: value pairs
mrd_pair_dict = {'uw' : ['Ux', 'Uz'],
                 'uv' : ['Ux', 'Uy'],
                 'uu' : ['Ux', 'Ux'],
                 'vw' : ['Uy', 'Uz'],
                 'vv' : ['Uy', 'Uy'],
                 'ww' : ['Uz', 'Uz'],
                 'utheta' : ['Ux', 'T_Sonic'],
                 'vtheta' : ['Uy', 'T_Sonic'],
                 'wtheta' : ['Uz', 'T_Sonic']
                 }

doy_test = doy_mrd(data_282, 72000, 282, mrd_pair_dict)
doy_test

# Runtime: 3798.5 sec

uw
uv
uu
vw
vv
ww
utheta
vtheta
wtheta


var                                                    uw  \
height                                               32.0   
values                                                  t   
0       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
1       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
2       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
3       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
4       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
5       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
6       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
7       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
8       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
9       [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
10      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
11      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
12      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
13      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
14      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
15      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
16      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
17      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
18      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
19      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
20      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
21      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
22      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   
23      [0.1, 0.2, 0.4, 0.8, 1.6, 3.2, 6.4, 12.8, 25.6...   

var                                                        \
height                                                      
values                                                  d   
0       [-8.161163330078883e-06, -8.154296874999996e-0...   
1       [3.536224365233907e-07, -1.3048171958008307e-0...   
2       [-1.0700225830078354e-05, -4.3290710449219e-05...   
3       [1.5517616271973462e-05, 6.314182281494107e-06...   
4       [-7.896423339845839e-07, 1.707077026367378e-07...   
5       [5.401229858398252e-06, 5.640602111816315e-06,...   
6       [1.8613815307616887e-06, -2.188014984130923e-0...   
7       [-8.155822753912323e-07, -7.645988464355317e-0...   
8       [5.828857539061252e-07, 4.61215973339846e-06, ...   
9       [1.1161422729492125e-05, 2.6005172729492323e-0...   
10      [-5.720138549805519e-06, -1.3439178466798211e-...   
11      [1.570243835449201e-05, 1.9127464294433827e-05...   
12      [-2.0355987548828294e-05, -4.688453674316397e-...   
13      [-2.704620361328177e-06, -2.5665283203125007e-...   
14      [-1.1564254760742182e-05, -3.3643531799317476e...   
15      [1.6304016113280314e-06, 3.15856933593749e-06,...   
16      [2.6059341430666298e-05, 2.6394462585449206e-0...   
17      [4.696788787841769e-05, 3.753843307495105e-05,...   
18      [2.1024896261232704e-05, -7.06738487810046e-05...   
19      [1.5720558557128716e-05, 1.569471298803709e-05...   
20      [6.950187712403821e-06, 8.748331067504868e-05,...   
21      [-6.978893275244206e-05, 4.1735935139404164e-0...   
22      [-2.2150039672849603e-05, 4.6656990051268746e-...   
23      [-1.696548461913973e-05, -9.919929504394556e-0...   

var                                                        \
height                                                      
values                                            d_accum   
0       [-8.161163330078883e-06, -1.631546020507888e-0...   
1       [3.536224365233907e-07, -9.5119475927744e-07, ...   
2       [-1.0700225830078354e-05, -5.3990936279297354e...   
3       [1.5517616271973462e-05, 2.1831798553467568e-0...   
4       [-7.896423339845839e-07, -6.18934631347846e-07...   
5       [5.401229858398252e-06, 1.1041831970214567e-05...   
6       [1.8613815307616887e-06, -3.2663345336923446e-...   
7       [-8.155822753912323e-07, -8.46157073974655e-06...   
8       [5.828857539061252e-07, 5

In [33]:
doy_test.to_csv('mrd_doy_282_numba.csv')

In [ ]:
test_day_282_1hr = pd.DataFrame(index=pd.date_range('2012-10-08T00:00', '2012-10-09T00:00', '1H'))
test_day_282_2hr = pd.DataFrame(index=pd.date_range('2012-10-08T00:00', '2012-10-09T00:00', '1H'))

test_day_279_1hr = pd.DataFrame(index=pd.date_range('2012-10-08T00:00', '2012-10-09T00:00', '1H'))

In [ ]:
asdf = pd.MultiIndex.from_product([pd.date_range('2012-10-08T00:00', '2012-10-09T00:00', '1H')])

In [ ]:
mrd_pair_dict = {'uw' : ['Ux', 'Uz'],
                 'uv' : ['Ux', 'Uy'],
                 'uu' : ['Ux', 'Ux'],
                 'vw' : ['Uy', 'Uz'],
                 'vv' : ['Uy', 'Uy'],
                 'ww' : ['Uz', 'Uz'],
                 'utheta' : ['Ux', 'T_Sonic'],
                 'vtheta' : ['Uy', 'T_Sonic'],
                 'wtheta' : ['Uz', 'T_Sonic']
                 }

In [ ]:
# AFTERNOON

mrd_pair_dict = {'utheta' : ['Ux', 'T_Sonic'],
                 'vtheta' : ['Uy', 'T_Sonic'],
                 'wtheta' : ['Uz', 'T_Sonic']
                 }

for k,v in mrd_pair_dict.items():
    print(k)

    n_heights = data_282['Ux'].columns.values.shape[0]
    fig, axes = plt.subplots(ncols=3, nrows=2, **{'figsize':(10,4), 'dpi':300})
    axes = axes.flatten()


    for i,height in enumerate(data_282['Ux'].columns.values):
        
        ax = axes[i]
        temp_data = data_282.loc[:,(slice(None),height)]

        # Calculate MRDs for each hour
        for hour in np.arange(19, 23):

            temp_hr_data = temp_data[temp_data.index.hour == hour].interpolate()
            temp_T, temp_d = mrd.mrd(temp_hr_data[v[0]].values, temp_hr_data[v[1]].values)

            ax.plot(temp_T*.05, temp_d, label=f"Hour={hour}")

        ax.set_title(f"Height: {height}", fontsize='small')

        ax.legend(fontsize='xx-small')
        ax.grid(ls='--', alpha=.7)
        ax.set_xscale('log')
        ax.set_xlim(1e-1, 1e4)
        ax.set_xlabel('Timescale [s]', fontsize='x-small')
        ax.tick_params(axis='both', which='major', labelsize='x-small')

    fig.tight_layout()
    fig.savefig(f'mrd_test_{k}_afternoon.png')

In [ ]:
mrd_pair_dict = {'uw' : ['Ux', 'Uz'],
                 'uv' : ['Ux', 'Uy'],
                 'uu' : ['Ux', 'Ux'],
                 'vw' : ['Uy', 'Uz'],
                 'vv' : ['Uy', 'Uy'],
                 'ww' : ['Uz', 'Uz'],
                 'utheta' : ['Ux', 'T_Sonic'],
                 'vtheta' : ['Uy', 'T_Sonic'],
                 'wtheta' : ['Uz', 'T_Sonic']
                 }

for k,v in mrd_pair_dict.items():
    print(k)

    n_heights = data_282['Ux'].columns.values.shape[0]
    fig, axes = plt.subplots(ncols=3, nrows=2, **{'figsize':(10,4), 'dpi':300})
    axes = axes.flatten()


    for i,height in enumerate(data_282['Ux'].columns.values):
        
        ax = axes[i]
        temp_data = data_282.loc[:,(slice(None),height)]

        # Calculate MRDs for each hour
        for hour in np.arange(7, 12):

            temp_hr_data = temp_data[temp_data.index.hour == hour].interpolate()
            temp_T, temp_d = mrd.mrd(temp_hr_data[v[0]].values, temp_hr_data[v[1]].values)

            ax.plot(temp_T*.05, temp_d, label=f"Hour={hour}")

        ax.set_title(f"Height: {height}", fontsize='small')

        ax.legend(fontsize='xx-small')
        ax.grid(ls='--', alpha=.7)
        ax.set_xscale('log')
        ax.set_xlim(1e-1, 1e4)
        ax.set_xlabel('Timescale [s]', fontsize='x-small')
        ax.tick_params(axis='both', which='major', labelsize='x-small')

    fig.tight_layout()
    fig.savefig(f'mrd_test_{k}_morning.png')

In [ ]:
import cupy as cp
print(cp.__version__)

In [6]:
## Testing out new way to store in netcdfs

# Initial run -- four days with raw data (at es4)

base_dir = 'C:/Users/moo90/Box/data/materhorn/ES'
n_hours = 1
n_out_vals = int(np.floor(np.log2(n_hours*72000)))
var_pair_dict = {
    'uu' : ['Ux', 'Ux'],
    'uv' : ['Ux', 'Uy'],
    'uw' : ['Ux', 'Uz'],
    'vv' : ['Uy', 'Uy'],
    'vw' : ['Uy', 'Uz'],
    'ww' : ['Uz', 'Uz'],
    'utheta' : ['Ux', 't_sonic'],
    'vtheta' : ['Uy', 't_sonic'],
    'wtheta' : ['Uz', 't_sonic']
}

coord_dict = {
    'station' : [f'ES{s}' for s in np.arange(1,6)],
    'date' : ['2012-10-01', '2012-10-05', '2012-10-08', '2013-05-25'],
    'hour' : np.arange(24),
    'var_pair' : list(var_pair_dict.keys()),
    'mrd_pt' : np.arange(n_out_vals)
}

data_dict = {   
    't' : (['mrd_pt'], np.zeros(n_out_vals)),
    'd' : (['mrd_pt'], np.zeros(n_out_vals)),
    'd_accum' : (['mrd_pt'], np.zeros(n_out_vals))
}

mrd_ds = xr.Dataset(data_dict, coord_dict)

In [11]:
mrd_ds

<xarray.Dataset>
Dimensions:   (date: 4, hour: 24, mrd_pt: 16, station: 5, var_pair: 9)
Coordinates:
  * station   (station) <U3 'ES1' 'ES2' 'ES3' 'ES4' 'ES5'
  * date      (date) <U10 '2012-10-01' '2012-10-05' '2012-10-08' '2013-05-25'
  * hour      (hour) int32 0 1 2 3 4 5 6 7 8 9 ... 14 15 16 17 18 19 20 21 22 23
  * var_pair  (var_pair) <U6 'uu' 'uv' 'uw' 'vv' ... 'utheta' 'vtheta' 'wtheta'
  * mrd_pt    (mrd_pt) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
Data variables:
    t         (mrd_pt) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    d         (mrd_pt) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    d_accum   (mrd_pt) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [19]:
t_da = xr.DataArray(np.ones(n_out_vals), 
coords={'station':'ES1', 'date':'2012-10-01', 'hour':2, 'var_pair':'uu', 'mrd_pt':np.arange(16)}, dims=['mrd_pt'])
mrd_ds.sel(station='ES1', date='2012-10-01', hour=2, var_pair='uu')['t'] = t_da

print(mrd_ds.sel(station='ES1', date='2012-10-01', hour=2, var_pair='uu')['t'])

<xarray.DataArray 't' (mrd_pt: 16)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
    station   <U3 'ES1'
    date      <U10 '2012-10-01'
    hour      int32 2
    var_pair  <U6 'uu'
  * mrd_pt    (mrd_pt) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15


In [21]:
print(t_da)

<xarray.DataArray (mrd_pt: 16)>
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
Coordinates:
    station   <U3 'ES1'
    date      <U10 '2012-10-01'
    hour      int32 2
    var_pair  <U2 'uu'
  * mrd_pt    (mrd_pt) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15


In [20]:
mrd_ds.sel(station='ES1', date='2012-10-01', hour=2, var_pair='uu')['t']

<xarray.DataArray 't' (mrd_pt: 16)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
    station   <U3 'ES1'
    date      <U10 '2012-10-01'
    hour      int32 2
    var_pair  <U6 'uu'
  * mrd_pt    (mrd_pt) int32 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15

In [16]:
for s in coord_dict['station']:
   
    for d in coord_dict['date']:
        
        # Read in station data
        f_date_str = d.replace('-', '_')
        try:
            in_path = os.path.join(base_dir, s, 'raw_20hz', 'parq', f'{s}_EC_20hz_{f_date_str}.parquet')
            temp_df = pd.read_parquet(in_path)
            print(print(f'Read data for station: {s} date: {d}'))
        except:
            print(f'No data for station: {s} date: {d}')
            continue

        # Run MRD for all hours in that day for all variables
        
        
    

No data for station: ES1 date: 2012-10-01
No data for station: ES1 date: 2012-10-05
No data for station: ES1 date: 2012-10-08
No data for station: ES1 date: 2013-05-25
Read data for station: ES2 date: 2012-10-01
None
C:/Users/moo90/Box/data/materhorn/ES\ES2\raw_20hz\parq\ES2_EC_20hz_2012_10_01.parquet
Read data for station: ES2 date: 2012-10-05
None
C:/Users/moo90/Box/data/materhorn/ES\ES2\raw_20hz\parq\ES2_EC_20hz_2012_10_05.parquet
Read data for station: ES2 date: 2012-10-08
None
C:/Users/moo90/Box/data/materhorn/ES\ES2\raw_20hz\parq\ES2_EC_20hz_2012_10_08.parquet
Read data for station: ES2 date: 2013-05-25
None
C:/Users/moo90/Box/data/materhorn/ES\ES2\raw_20hz\parq\ES2_EC_20hz_2013_05_25.parquet
Read data for station: ES3 date: 2012-10-01
None
C:/Users/moo90/Box/data/materhorn/ES\ES3\raw_20hz\parq\ES3_EC_20hz_2012_10_01.parquet
Read data for station: ES3 date: 2012-10-05
None
C:/Users/moo90/Box/data/materhorn/ES\ES3\raw_20hz\parq\ES3_EC_20hz_2012_10_05.parquet
Read data for station: